# 7일차: PyTorch 모델 구성 및 데이터 로딩

## 학습 목표
- nn.Module로 신경망 모델 정의
- 손실 함수와 옵티마이저 사용
- 완전한 학습 루프 구현
- Dataset과 DataLoader 활용

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

print(f'PyTorch 버전: {torch.__version__}')

## �� 이론 개요

### nn.Module
PyTorch의 모든 신경망은 `nn.Module`을 상속
- `__init__()`: 레이어 정의
- `forward()`: Forward pass 정의
- `parameters()`: 학습 가능한 파라미터 자동 관리

### 학습 루프 4단계
1. **Forward pass**: 예측값 계산
2. **Loss 계산**: 예측과 정답 비교
3. **Backward pass**: Gradient 계산
4. **Parameter 업데이트**: Optimizer로 가중치 갱신

### Dataset & DataLoader
- `Dataset`: 데이터와 라벨 반환
- `DataLoader`: 배치 단위로 데이터 제공, 섞기, 병렬 로딩

## 📚 1. nn.Module 기본

### 1.1 간단한 선형 모델

In [ ]:
# nn.Module 상속하여 모델 정의
class LinearModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.linear(x)

# 모델 생성
model = LinearModel(input_dim=1, output_dim=1)
print('모델 구조:')
print(model)
print()

# 파라미터 확인
for name, param in model.named_parameters():
    print(f'{name}: {param.shape}')
    print(param)

### 1.2 다층 신경망 (MLP)

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

mlp = MLP(input_dim=10, hidden_dim=20, output_dim=2)
print('MLP 구조:')
print(mlp)
print()

# 테스트 입력
test_input = torch.randn(5, 10)  # 배치 5, 특성 10
output = mlp(test_input)
print(f'입력 shape: {test_input.shape}')
print(f'출력 shape: {output.shape}')

## 📚 2. 손실 함수와 옵티마이저

### 2.1 손실 함수

In [ ]:
# MSE Loss (회귀)
mse_loss = nn.MSELoss()
pred = torch.tensor([1.0, 2.0, 3.0])
target = torch.tensor([1.5, 2.5, 2.5])
loss = mse_loss(pred, target)
print(f'MSE Loss: {loss.item():.4f}')
print(f'계산: mean((pred - target)²) = mean([0.25, 0.25, 0.25]) = 0.25')
print()

# CrossEntropy Loss (분류)
ce_loss = nn.CrossEntropyLoss()
logits = torch.randn(3, 5)  # 3개 샘플, 5개 클래스
labels = torch.tensor([1, 0, 4])  # 정답 레이블
loss = ce_loss(logits, labels)
print(f'CrossEntropy Loss: {loss.item():.4f}')

### 2.2 옵티마이저

In [ ]:
model = LinearModel(1, 1)

# SGD
optimizer_sgd = optim.SGD(model.parameters(), lr=0.01)
print('SGD Optimizer:')
print(optimizer_sgd)
print()

# Adam (더 많이 사용됨)
optimizer_adam = optim.Adam(model.parameters(), lr=0.001)
print('Adam Optimizer:')
print(optimizer_adam)

## 📚 3. 완전한 학습 루프

### 3.1 데이터 준비

In [ ]:
# 선형 데이터 생성: y = 3x + 1 + noise
torch.manual_seed(42)
x_train = torch.randn(100, 1)
y_train = 3 * x_train + 1 + 0.2 * torch.randn(100, 1)

print(f'훈련 데이터: x {x_train.shape}, y {y_train.shape}')
print(f'목표: y = 3x + 1을 학습')

### 3.2 모델 학습

In [ ]:
# 모델, 손실, 옵티마이저
model = LinearModel(1, 1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 100

for epoch in range(num_epochs):
    # 1. Forward pass
    y_pred = model(x_train)
    
    # 2. Loss 계산
    loss = criterion(y_pred, y_train)
    
    # 3. Backward pass
    optimizer.zero_grad()  # Gradient 초기화
    loss.backward()         # Gradient 계산
    
    # 4. Parameter 업데이트
    optimizer.step()
    
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print()
print('학습된 파라미터:')
for name, param in model.named_parameters():
    print(f'{name}: {param.data.numpy()}')

print('\n목표: weight=3, bias=1')

## 📚 4. Dataset과 DataLoader

### 4.1 커스텀 Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Dataset 생성
dataset = CustomDataset(x_train, y_train)
print(f'Dataset 크기: {len(dataset)}')
print(f'첫 번째 샘플: {dataset[0]}')

### 4.2 DataLoader 사용

In [ ]:
# DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

print(f'총 배치 수: {len(dataloader)}')
print()

# 배치 순회
for batch_idx, (batch_x, batch_y) in enumerate(dataloader):
    print(f'Batch {batch_idx+1}: x {batch_x.shape}, y {batch_y.shape}')
    if batch_idx == 2:  # 처음 3개만 출력
        print('...')
        break

### 4.3 DataLoader로 학습

In [ ]:
# 새 모델
model = LinearModel(1, 1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 50

for epoch in range(num_epochs):
    epoch_loss = 0
    
    for batch_x, batch_y in dataloader:
        # Forward
        y_pred = model(batch_x)
        loss = criterion(y_pred, batch_y)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        avg_loss = epoch_loss / len(dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}')

print('\n배치 학습 완료!')

## 📚 5. 이진 분류 예제

### 5.1 데이터 생성

In [ ]:
# 2D 점 분류 (x2 > x1이면 클래스 1)
torch.manual_seed(42)
x_data = torch.randn(200, 2)
y_data = (x_data[:, 1] > x_data[:, 0]).long()  # 0 또는 1

print(f'데이터: {x_data.shape}')
print(f'라벨: {y_data.shape}')
print(f'클래스 0: {(y_data == 0).sum()}, 클래스 1: {(y_data == 1).sum()}')

### 5.2 분류 모델

In [ ]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 10)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(10, 2)  # 2개 클래스
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x  # Logits (CrossEntropyLoss가 softmax 포함)

model = BinaryClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

print(model)

### 5.3 학습 및 평가

In [ ]:
# Dataset & DataLoader
dataset = CustomDataset(x_data, y_data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    correct = 0
    total = 0
    
    for batch_x, batch_y in dataloader:
        # Forward
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
        # 정확도 계산
        _, predicted = torch.max(outputs, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
    
    if (epoch + 1) % 20 == 0:
        accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(dataloader):.4f}, Accuracy: {accuracy:.2f}%')

print('\n학습 완료!')

## 🔥 연습 문제

### 문제 1: 회귀 모델 구현

In [ ]:
# TODO: y = 2x₁ + 3x₂ + 1를 학습하는 모델 만들기
# 1. 데이터 생성
# 2. 모델 정의
# 3. 학습

# 여기에 코드 작성


In [ ]:
# 해답
torch.manual_seed(42)
x = torch.randn(100, 2)
y = 2*x[:, 0:1] + 3*x[:, 1:2] + 1 + 0.1*torch.randn(100, 1)

model = LinearModel(2, 1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(200):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 50 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

print('\n학습된 가중치:')
print(model.linear.weight.data)
print('목표: [2, 3]')
print('\n편향:')
print(model.linear.bias.data)
print('목표: 1')

## 📝 7일 학습 완료!

### 배운 내용 총정리

**Day 1-3: NumPy 기초**
- 배열 조작, 차원 변환
- 브로드캐스팅, 행렬 연산
- 집계, 필터링

**Day 4-5: 선형대수**
- 역행렬, 행렬식, 랭크
- SVD, 고유값/고유벡터
- 유사역행렬, PSD

**Day 6-7: PyTorch**
- Tensor, Autograd
- nn.Module, nn.Linear
- Loss, Optimizer
- Dataset, DataLoader

### 다음 단계
1. CNN, RNN 등 고급 아키텍처
2. 실제 데이터셋으로 실습
3. GPU 학습, 모델 저장/로드
4. Transfer Learning

**축하합니다! 🎉**
딥러닝 기초를 완성했습니다!